### Homework on Classification

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#### Read the data source

data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

!wget $data -O bank_data.csv

In [28]:
df = pd.read_csv('bank_data.csv')

In [29]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


#### Data Preparation
- Check if the missing values are presented in the features.
- If there are missing values:
    - For caterogiral features, replace them with 'NA'
    - For numerical features, replace with with 0.0

In [30]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

##### Find the numeric data columns

In [31]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [32]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']

In [33]:
numeric = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score', 'converted']

In [34]:
df[categorical] = df[categorical].fillna('NA')

In [35]:
df[numeric] = df[numeric].fillna(0)

In [36]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

#### Question 1
What is the most frequent observation (mode) for the column industry?

In [37]:
df.industry.mode()

0    retail
Name: industry, dtype: object

#### Question 2
Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

- interaction_count and lead_score
- number_of_courses_viewed and lead_score
- number_of_courses_viewed and interaction_count
- annual_income and interaction_count
- Only consider the pairs above when answering this question.

In [38]:
df[numeric].dtypes

number_of_courses_viewed      int64
annual_income               float64
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [43]:
correlation_matrix = df[numeric].corr()
correlation_matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


#### Split the data
- Split your data in train/val/test sets with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
- Make sure that the target value y is not in your dataframe.

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [46]:
len(df_full_train), len(df_test)

(1169, 293)

In [47]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [48]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [49]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [50]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [52]:
y_train[:10]

array([0, 1, 1, 1, 1, 1, 0, 1, 0, 0])

In [53]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

#### Question 3
- Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
- Round the scores to 2 decimals using round(score, 2).

Which of these variables has the biggest mutual information score?
- industry
- location
- lead_source
- employment_status

In [55]:
from sklearn.metrics import mutual_info_score

In [56]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.converted)

In [57]:
df_full_train[categorical].apply(mutual_info_churn_score).sort_values(ascending=False)

lead_source          0.025665
employment_status    0.013258
industry             0.011685
location             0.002253
dtype: float64